In [16]:
using DataFrames #for data wrangling
using StatsPlots #for plotting
using Turing #for MCMC
using CSV #CSV import
using Random #data generation (if required)

ArgumentError: ArgumentError: Package DataFrames not found in current path.
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.

Import Data

In [17]:
#this is a local document, but i've also saved a slightly cleaned up version in github with the same "valveTestRaw" title
valveData = CSV.read("./Results of New Valve Tests.csv", DataFrame)

UndefVarError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Collect Number of Trials & Separate into Service Mediums

In [18]:
# vendors = (collect(valveData[:,1]))
# n=0
# for i in 1:length(vendors)
#     n = valveData[i, 2] + n
# end
# n

steamDF = []
airDF = []
liquidDF = []
airLiqDF = []

steamDF = filter(:Service => ==("steam"), valveData)
airDF = filter(:Service => ==("air"), valveData)
liquidDF = filter(:Service => ==("liquid"), valveData)
airLiqDF = filter(:Service => ==("air/liquid"), valveData)
naDF = filter(:Service => ==("na"), valveData)

n_steam = 0
n_air = 0
n_liquid = 0
n_airLiq = 0
n_na = 0

steamVend = (collect(steamDF[:,1]))
for i in 1:length(steamVend)
    n_steam = steamDF[i,2] +n_steam
end

airVend = (collect(airDF[:,1]))
for i in 1:length(airVend)
    n_air = airDF[i,2] +n_air
end

liquidVend = (collect(liquidDF[:,1]))
for i in 1:length(liquidVend)
    n_liquid = liquidDF[i,2] +n_liquid
end

airLiqVend = (collect(airLiqDF[:,1]))
for i in 1:length(airLiqVend)
    n_airLiq = airLiqDF[i,2] +n_airLiq
end

naVend = (collect(naDF[:,1]))
for i in 1:length(naVend)
    n_na = naDF[i,2]+n_na
end

steamDF

UndefVarError: UndefVarError: `valveData` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Set Up Model

In [ ]:
@model function valveTesting(Vendors, Population, highPops) # rejects serviceMedium)
    #additional functionality can be added to this, especially when considering that the probability of failing (p-1) is the sum of the probability of failing on a low pop (less dangerous) & a high pop (more dangerous)      
    
    
    #hyper prior
        μ ~ Beta(4, 2)
        σ ~ Exponential(1)
    
        p = Vector{Real}(undef, Vendors)
        n = Vector{Real}(undef, Vendors)
    for i in 1:length(Vendors)
        
        #prior
        n[i] = Population[1, i]
        p[i] ~ Normal(μ, σ)
        #distribution of valves passing given pass probability p and n trials
        valvePass[i] ~ Binomial(n, p[i])
    end
end

LoadError: LoadError: UndefVarError: `@model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at c:\Users\flemm\Documents\Grad School\MAE 298 (Bayesian Statistics)\Homework\Final Project\Project Git\ValveRejectionRates\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:1

In [20]:
steam_model = valveTesting(steamDF[:,1], steamDF[:,2], steamDF[:,3])

steam_posterior = sample(steam_model, NUTS(), 1000)

UndefVarError: UndefVarError: `valveTesting` not defined in `Main`
Suggestion: check for spelling errors or missing imports.